## Collections Class Hierarchy
Hierarchy (excluding abstract classes):
```
Collection (I)
            +-- Set (I)
            |        +-- HashSet (C)
            |        |            +-- LinkedHashSet (C)
            |        +-- SortedSet (I)
            |                       +-- NavigableSet (I)
            |                                         +-- TreeSet (C)
            +-- List (I)
            |         +-- ArrayList (C)
            |         +-- Vector (C)
            |         +-- LinkedList (C) also implements Deque
            +-- Queue (I)
                       +-- Deque (I)
                       |          +-- LinkedList (C) also implements List
                       +-- PriorityQueue (C)
```

## List
`ArrayList` is the most used List implementation. `Vector` is same as ArrayList, except that it is synchronized. Both of them internally use an `Object[]` array. All the addition, removal and traversal happens on this array. The array is grown dynamically.

In [1]:
List<Float> someValues = new ArrayList<>();
someValues.add(2.5f);
someValues.add(4.7f);
for(float f: someValues){
    System.out.println(f);
}

2.5
4.7


## Set
`HashSet` is the most common implementation of Set. Internally it uses a map to store information. The values added to the set forms the map's keys, whereas the map's values are set as null.  

None of the Set implementations as synchronous. Use `Collections.synchronizedSet()`.

In [2]:
Set<Integer> primes = new HashSet<>();
primes.add(2);primes.add(2);primes.add(3);
primes.add(5);primes.add(7);primes.add(7);

System.out.println(primes.size())

4


Since, HashSet is based on map, its iteration order is not the same as that of insertion order. To maintain insertion order, use `LinkedHashSet`.

In [6]:
Iterator<Integer> pIterator = primes.iterator();
while(pIterator.hasNext())
    System.out.print(pIterator.next() + " ");
    
Set<Integer> anotherPrimes = new LinkedHashSet<>();
anotherPrimes.add(2);anotherPrimes.add(2);anotherPrimes.add(3);
anotherPrimes.add(5);anotherPrimes.add(7);anotherPrimes.add(7);

System.out.println();
Iterator<Integer> apIterator = anotherPrimes.iterator();
while(apIterator.hasNext())
    System.out.print(apIterator.next() + " ");

2 3 5 7 
2 3 5 7 

To maintain natural sorting order, use `TreeSet`.

In [7]:
Set<Integer> randomNumbers = new TreeSet<>();
for(int i=0; i<5; i++){
    Random r = new Random();
    randomNumbers.add(r.nextInt());
}

for(int j: randomNumbers)
    System.out.print(j + " ");

124817519 152547569 390882712 879733970 1702448752 

## The Collections Class
Similar to `Arrays`, class `Collections` provides several useful utility functions to perform various operations on collections. Consider class as defined below:

In [8]:
public class Country implements Comparable<Country>{
    String name;
    int pop;
    float area;
    
    Country(String name, int pop, float area){
        this.name = name;
        this.pop = pop;
        this.area = area;
    }
    
    @Override
    public int compareTo(Country c){
        return this.pop - c.pop;
    }
    
    @Override
    public String toString(){
        return name + "[pop = " + pop + ", area = " + area + "]";
    }
}

Before we know how to sort collections, we must know about `Comparable` and `Comparator`. In order to use `Arrays.sort` or `Collections.sort` methods, the object in array or collection must implement `Comparable` interface thereby override `compareTo` method. Collection/array of primitive data types do not have this requirement.  
If we want to sort by some other property of the object, we can supply a `Comparator` object to the sort methods.

In [10]:
List<Country> countries = new ArrayList<>();
countries.add(new Country("India", 1300, 3.28f)); countries.add(new Country("China", 1400, 9.68f));
countries.add(new Country("Bangladesh", 140, 0.64f)); countries.add(new Country("USA", 300, 9.7f));
Collections.sort(countries);    // Sorted by population
for(Country country: countries){
    System.out.println(country);
}

Bangladesh[pop = 140, area = 0.64]
USA[pop = 300, area = 9.7]
India[pop = 1300, area = 3.28]
China[pop = 1400, area = 9.68]


In [ ]:
// If we want to sort by area instead, we need to use Comparator
Collections.sort(countries, (c1, c2)->{
    return (int)(c1.area - c2.area);
});

/*
Equivalently
Collections.sort(countries, new Comparator<Country>(){
    @Override
    public int compare(Country c1, Country c2){
        return c1.area - c2.area;
    }
});
*/

for(Country country: countries){
    System.out.println(country);
}

## Map Class Hierarchy
```
Map (I)
      +-- HashMap (C) non synch
      |             +-- LinkedHashMap (C)
      +-- HashTable (C) synch
      +-- SortedMap (I)
                      +-- NavigableMap (I)
                                         +-- TreeMap (C)
```

**Equals and Hash Contract:** Hash retrieval is a two step process
- find the righ bucket (using `hashCode`)
- search the bucket for the right element (using `equals`)

Whenever we override the equals method we should also override hashCode method. Suppose we override equals, but we do not override hashCode. In this case equal object can be put in different buckets. On the other hand, if we override hashCode but not equals then equal objects will be placed in same bucket but no replacement will occur.


**Objects as keys:** it is better to override `hashCode` and `equals` methods for an object which are intended as keys. It is perfectly legal not to do so though. The hashcode helps to identify the bucket whereas equals method hellps in searching through the bucket.  
Changing the object after it has been added to a Map may change its hashcode (if it is implemented that way). Once its hashcode has changed, we may not be able to search it in the map.

## HashMap

In [2]:
Map<String, Integer> numbers = new HashMap<>();
numbers.put("First", 1); numbers.put("Second", 2); numbers.put("Third", 3);
numbers.put("Fourth", 4); numbers.put("Fifth", 5); numbers.put("Sixth", 6);

// The keys are printed in a different order than insertion
for(String key: numbers.keySet()){
    System.out.println(key + " : " +numbers.get(key));
}

Second : 2
Sixth : 6
Third : 3
First : 1
Fourth : 4
Fifth : 5


**HashMap vs HashTable:**
- HashMap is not synchronized, HashTable is
- HashMap can contain one null ke and as manu null values. HashTable cannot have either null key or null value

Better use `ConcurrentHashMap` instead of HashTable

**Internal working of HashMap:** HashMap in Java use seperate chaining based approach. There is a Node object which contains the following data:
```java
class Node <K,V>{
    final int hash;
    final K key;
    V value;
    Node <K,V> next;
}
```

Bucket is represented as array of nodes
```java
Node<K,V>[] table; 
```

Whenever an item is put into the HashMap,
```java
HashMap<String, Integer> map = new HashMap<>();
map.put("Aa", 32);
```

The HashMap calls its internal hash method which in turn calls `String.hashCode` in this case. If the hash is greater than `table.length`, then the appropriate bucket index is obtained by:
```java
// Hashmap size is always of the order 2^n, 
// and doing a bitwise operation on such values 
// results same as a modulo operations but faster.
int index = key.hashCode() & (table.length - 1);
```

If `table[index]` is null,
```java
Node<String, Integer> node = new Node<>(hash, key, value, null);
table[index] = node;
```

When collision occurs,
i) A replacement occurs (replacement of values property of Node) if `newKey.equals(oldKey) == true`  
ii) Else, a new node is added to the existing linked list  

Average time complexity of `put` is $O(1)$ and in the worst case it is $O(n)$  

**Initial capacity and Load Factor**
- Initial Capacity is a measure used internally by HashMap that represents the number of bucket or size of the bucket array at the time of creation of HashMap. Default capacity is $2^4 = 16$   
- Load Factor is a factor that is used by HashMap internally to decide when the size of Bucket array needs to be increased. It is 0.75 by default i.e., when the number of nodes in the HashMap is more than 75% of Total Capacity then the HashMap grows its bucket array size. Bucket size is doubled.

**Java 8 Changes:** How HashMap works internally in Java 8 is a little bit different from prior versions of java. HashMap in Java 8, maintains a value called `TREEIFY_THRESHOLD`, it is an integer constant and currently the value of `TREEIFY_THRESHOLD` is 8. It is used as whenever in any bucket the number of nodes becomes more than this threshold value, the data structure of that bucket is converted from linked-list to balanced tree. This change is done to make put-get operations faster as the linked-list provides $O(n)$ time complexity whereas the time required to traverse the balance tree is $O(log n)$ .

## Properties
The `Properties` class extends `HashTable`.

In [5]:
// Properties can be used as normal Map
Properties capitals = new Properties();
capitals.put("Italy", "Rome");
capitals.put("France", "Paris");
capitals.put("Germany", "Berlin");
capitals.put(2,2);  // Can even have integer as key even though this isn't the intended purpose

System.out.println("Capital of France is " + capitals.get("France"));  // get method discouraged, use getProperty
System.out.println("Capital of Italy is " + capitals.getProperty("Italy"));

Capital of France is Paris
Capital of Italy is Rome


In [ ]:
// The main reason why Property is used is because it can save key-value pairs to streams
capitals.store(new OutputStreamWriter(new FileOutputStream(new File("capitals.properties"))));

Property capitalsAgain = new Property();
capitalsAgain.load(new InputStreamWriter(new FileInputStream(new File("capitals.properties"))));